In [1]:
#!pip install numpy
#!pip install sklearn

# Imports of useful packages
import matplotlib # For the plots
import matplotlib.pyplot as plt 
import numpy as np # To perform operations on matrices efficiently


# We will use the sklearn library to compare our neural network to that 
# of a simpler approach like logistic regression

import sklearn 
import sklearn.datasets
import sklearn.linear_model

from math import exp,log

# To display plots inline and adjust the display

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

In [2]:
np.random.seed(1)
X, y = sklearn.datasets.make_moons(n_samples=300, noise=0.20)  # We create a dataset with 300 elements

In [3]:
#### display function
def plot_decision_boundary(pred_func):
    """
    Shows the decision boundaries of a binary prediction function.
    """
    # Set grid dimensions and give some margin for display
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    h = 0.01
    # Generate the grid of points with a distance of h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Drawing the decision boundary
    Z = pred_func(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Show contour and training points
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Spectral)

In [4]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x)*(1-sigmoid(x))

def softmax(Z):
    exp_scores = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

def one_hot(y):
    one_hot_y = np.zeros((y.size, y.max() + 1))
    one_hot_y[np.arange(y.size), y] = 1
    return one_hot_y


In [5]:
class MLP(object):
    def __init__(self, activation_function_l1, learning_rate, lambd, num_epochs, d_input, d_hidden, d_output):
        self.lr = learning_rate
        self.num_epochs = num_epochs
        np.random.seed(0)    
        self.activation_function_l1 = activation_function_l1
        self.W1 = np.random.rand(d_input, d_hidden)-0.5 
        self.b1 = np.random.rand(d_hidden)-0.5 
        self.W2 = np.random.rand(d_hidden, d_output)-0.5 
        self.b2 = np.random.rand(d_output)-0.5 
        self.best_model_iteration = 0
        self.lambd = lambd # Weight decay
        
    def update_params(self, W1, b1, W2, b2, best_model_iteration):
        self.W1 = W1
        self.b1 = b1 
        self.W2 = W2
        self.b2 = b2
        self.best_model_iteration = best_model_iteration
        
    def forward_function(self, X):
        z1 = X.dot(self.W1) + self.b1.T # Output of the first layer
        a1 = activation_function_l1(z1) # Sigmoid activation of the first layer
        z2 = a1.dot(self.W2) + self.b2.T # Output of the second layer
        probs = softmax(z2) #Apply softmax activation function on z2
        return z1, a1, z2, probs
    
    def backpropagation(self, z1, a1, z2, probs, X, y):
        delta2 = probs - one_hot(y)
        dW2 = a1.T.dot(delta2)
        db2 = np.sum(delta2, axis=0)
        delta1 = sigmoid_prime(z1) * delta2.dot(self.W2.T)
        dW1 = X.T.dot(delta1)
        db1 = np.sum(delta1, axis=0)
        return dW1, db1, dW2, db2

    def gradient_descent(self, dW1, db1, dW2, db2):
        self.W1 = self.W1 - self.lr*self.lambd*self.W1 - self.lr * dW1
        self.b1 = self.b1 - self.lr*self.lambd*self.b1 - self.lr * db1
        self.W2 = self.W2 - self.lr*self.lambd*self.W2 - self.lr * dW2
        self.b2 = self.b2 - self.lr*self.lambd*self.b2 - self.lr * db2
    
    def predict(self, X):
        _, _, _, probs = self.forward_function(X)
        return np.argmax(probs, axis=1)
    
    def get_predictions(self, probs):
        return np.argmax(probs, axis=1)
    
    def accuracy(self, y, y_pred):
        return np.average(y == y_pred)                        
    
    def estimate_loss(self, y, probs):
        correct_logprobs = -np.log(probs[np.arange(len(probs)), y]) # Calculation of cross entropy for each example
        return np.average(correct_logprobs) # Total loss
          
    def fit(self, X, y, print_loss=False, return_best_model=True):
        z1, a1, z2, probs = self.forward_function(X) # this will help us not compute the accuracy two times in the loop.
        # best model
        bm = {'W1': self.W1, 'b1': self.b1, 'W2': self.W2, 'b2': self.b2, 
                      'accuracy': self.accuracy(y, self.get_predictions(probs)), 'iteration': -1}
        
        for i in range(0, self.num_epochs):
            dW1, db1, dW2, db2 = self.backpropagation(z1, a1, z2, probs, X, y)
            self.gradient_descent(dW1, db1, dW2, db2) # update parameters
            z1, a1, z2, probs = self.forward_function(X)
    
            current_accuracy = self.accuracy(y, self.get_predictions(probs)) # predictions of epoch

            if print_loss and i % 100 == 0:
                print("Loss at epoch %i: %f" %(i, self.estimate_loss(y, probs)), "Accuracy :", current_accuracy)
                
            if(return_best_model and current_accuracy > bm['accuracy']):
                bm = {'W1': self.W1, 'b1': self.b1, 'W2': self.W2, 'b2': self.b2, 
                              'accuracy': self.accuracy(y, self.get_predictions(probs)), 'iteration': i}
        
        if(print_loss and return_best_model):
            self.update_params(bm['W1'], bm['b1'], bm['W2'], bm['b2'], bm['iteration'])
            print("The best accuracy obtained is :", model.accuracy(y, model.predict(X)), ", at this iteration :", model.best_model_iteration)
        elif(print_loss):
            print("The model accuracy obtained is :", model.accuracy(y, model.predict(X)))

In [6]:
def shuffle_and_split_data(X, y, train_percentage):
    N = len(X)
    train_size = int(N * train_percentage) # 80% of samples for training
    test_size = N - train_size # 20% remaining for test
    p = np.random.permutation(N) # shuffle X and y with the same random permutation. 
    X = X[p]
    y = y[p]

    # split data
    X_train = X[0:train_size]
    y_train = y[0:train_size]
    
    X_test = X[train_size:N]
    y_test = y[train_size:N]
    return X_train, y_train, X_test, y_test

In [7]:
X_train, y_train, X_test, y_test = shuffle_and_split_data(X, y, 0.8)

In [8]:
activation_function_l1=sigmoid
learning_rate = 3e-2
lambd=0.0
num_epochs=200
d_input=2 
d_hidden=10 
d_output=2


model = MLP(activation_function_l1, learning_rate, lambd, num_epochs, d_input, d_hidden, d_output)
model.fit(X_train, y_train, print_loss=True, return_best_model=False)
print("The test accuracy obtained is :", model.accuracy(y_test, model.predict(X_test)))

Loss at epoch 0: 1.308488 Accuracy : 0.49166666666666664
Loss at epoch 100: 0.149351 Accuracy : 0.9625
The model accuracy obtained is : 0.9708333333333333
The test accuracy obtained is : 0.9333333333333333


In [9]:
import ctypes

In [ ]:
def cuda_sigmoid(x):
    # Load the shared library
    mylibrary = ctypes.CDLL('./sigmoid.so')

    # Define the function signature

    my_library.sigmoid_of_matrix.argtypes = [ctypes.POINTER(ctypes.c_float), ctypes.c_int, ctypes.c_int]
    mylibrary.sigmoid_of_matrix.restype = ctypes.c_float

    # Get a pointer to the data
    x_ptr = x.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

    # Call the CUDA function with the pointer
    result = mylibrary.sigmoid_of_matrix(x_ptr, len(x), len(x[0]))

    return result


In [ ]:
# Get a pointer to the data
data_ptr = data.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

# Call the CUDA function with the pointer
result = mylibrary.my_function(data_ptr, len(data))

In [ ]:
activation_function_l1=sigmoid
learning_rate = 3e-2
lambd=0.0
num_epochs=200
d_input=2 
d_hidden=10 
d_output=2


model = MLP(activation_function_l1, learning_rate, lambd, num_epochs, d_input, d_hidden, d_output)
model.fit(X_train, y_train, print_loss=True, return_best_model=False)
print("The test accuracy obtained is :", model.accuracy(y_test, model.predict(X_test)))